In [1]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 147kB/s  eta 0:00:01   |█████▏                          | 35.3MB 6.6MB/s eta 0:00:28��        | 159.5MB 43.8MB/s eta 0:00:02MB/s eta 0:00:02
     |████████████████████████████████| 204kB 37.6MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# delete files from previous runs
!rm -f hmp.parquet*

# download the file containing the data in PARQUET format
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

--2020-06-27 03:24:33--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2020-06-27 03:24:33--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2020-06-27 03:24:33--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving 

In [4]:
df.show()

+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [5]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler,Normalizer
from pyspark.ml.linalg import Vectors

In [7]:
#preprocessing
indexer = StringIndexer(inputCol="class",outputCol="label")
encoder = OneHotEncoder(inputCol="label", outputCol="labelVec")
vA = VectorAssembler(inputCols=["x","y","z"],outputCol="features")
normalizer = Normalizer(inputCol="features",outputCol="features_norm",p=1.0)

In [8]:
df.createOrReplaceTempView('df')
df_class = spark.sql("select * from df where class in ('Use_telephone','Standup_chair')")


In [9]:
#for binary classification
splits = df_class.randomSplit([0.8,0.2])
df_train = splits[0]
df_test = splits[1]

In [11]:
#gradient boosting
from pyspark.ml.classification import GBTClassifier
GB = GBTClassifier(labelCol="label",featuresCol="features",maxIter=10)

In [12]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer,encoder,vA,normalizer,GB])

In [13]:

#training
model = pipeline.fit(df_train)

In [14]:
pred = model.transform(df_train)

In [16]:
#evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("pred").setLabelCol("label")
binEval.evaluate(pred)

IllegalArgumentException: 'Field "pred" does not exist.\nAvailable fields: x, y, z, source, class, label, labelVec, features, features_norm, rawPrediction, probability, prediction'